## Importações

In [13]:
import pandas as pd
import os
# Obtém o diretório de trabalho atual
current_dir = os.getcwd()

In [14]:
import os
import requests
import zipfile
import io

# URL do arquivo ZIP
url = "https://archive.ics.uci.edu/static/public/502/online+retail+ii.zip"

# Nome do arquivo ZIP
filename = "online+retail+ii.zip"

# Caminho de destino para salvar e extrair o arquivo ZIP
destination_path = os.path.join(os.getcwd(), "raw-data")

# Verifica se o arquivo já existe no destino
file_path = os.path.join(destination_path, filename)
if os.path.exists(file_path):
    print("O arquivo já existe no destino. Nenhuma cópia necessária.")
else:
    # Realiza o download do arquivo ZIP
    print("Baixando o arquivo ZIP...")
    response = requests.get(url)
    response.raise_for_status()

    # Salva o conteúdo do arquivo ZIP em disco
    with open(file_path, "wb") as file:
        file.write(response.content)

    # Extrai o conteúdo do arquivo ZIP
    print("Extraindo o arquivo ZIP...")
    with zipfile.ZipFile(file_path, "r") as zip_ref:
        zip_ref.extractall(destination_path)

    print("Extração concluída.")


O arquivo já existe no destino. Nenhuma cópia necessária.


### Importando o arquivo:

In [11]:
# Caminho para o arquivo Excel
filepath =  current_dir + "/raw-data/online_retail_II.xlsx"

# Lendo o arquivo Excel usando o Pandas
df = pd.read_excel(filepath)

## Informações dos atributos:


**InvoiceNo**: Número da fatura. (Nominal) Um número inteiro de 6 dígitos atribuído exclusivamente a cada transação. Se o código começar com a letra 'c', indica um cancelamento  
**StockCode**: Código do produto (item). (Nominal) Um número inteiro de 5 dígitos atribuído exclusivamente a cada produto distinto.  
**Description**: Descrição do produto (item). (Nominal)  
**Quantity**: Quantidade de cada produto (item) por transação. (Numérico)  
**InvoiceDate**: Data e hora da fatura. (Numérico) O dia e a hora em que a transação foi gerada.  
**UnitPrice**: Preço unitário. (Numérico) Preço do produto por unidade em libras esterlinas (£).  
**CustomerID**: Número do cliente. (Nominal) Um número inteiro de 5 dígitos atribuído exclusivamente a cada cliente.  
**Country**: País. (Nominal) O nome do país onde o cliente reside.  

In [12]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
